# Working with neuroimaging data in Python

We've already learned about general Python tools, but there is also a growing ecosystem of Python-based tools for working with neuroimaging data.

In this training, we will focus on `pybids`, `nibabel`, and `nilearn`.

In [ ]:
%matplotlib inline
import os.path as op  # for navigating the system
from pprint import pprint  # for pretty-printing dictionaries
import numpy as np

dataset_folder = '/scratch/madlab/condassoc_bids/'

## PyBIDS
[PyBIDS](https://bids-standard.github.io/pybids/) is a package for **working with BIDS datasets**.

With PyBIDS, you can:
1. Validate datasets to make sure they're BIDS compliant.
2. Search datasets for specific files.
3. Read in metadata for files within datasets.
4. Generate citable reports summarizing datasets.
5. Apply transformations to variables in datasets for analysis.

In this training we will cover 1-3.

In [ ]:
from bids import BIDSLayout

In [ ]:
# Load in BIDS dataset as BIDSLayout object
layout = BIDSLayout(dataset_folder, validate=True)

In [ ]:
# First let's look at the dataset description file to learn about the dataset
pprint(layout.get_dataset_description())

In [ ]:
# You can use the BIDSLayout object to search the data
# for different kinds of information
subjects = layout.get_subjects()
print('{} subjects in dataset'.format(len(subjects)))
print('Subject IDs: {}'.format(', '.join(subjects)))
print()

tasks = layout.get_tasks()
print('{} tasks in dataset'.format(len(tasks)))
print('Tasks: {}'.format(', '.join(tasks)))
print()

data_types = layout.get_datatypes()
print('{} data types in dataset'.format(len(data_types)))
print('Data types: {}'.format(', '.join(data_types)))

In [ ]:
# If you're confused about what you can search for,
# check the 'entities' attribute
# Each entity can be search with a "get_<entity>s" function
layout.entities

In [ ]:
# Each search function can take other entities as arguments
# For example, what repetition times exist in the dataset?
print(layout.get_RepetitionTimes())

# Ah, but what about for functional scans only?
print(layout.get_RepetitionTimes(datatype='func'))

In [ ]:
# There is also the general search method `get` that returns BIDS-type file object
sub_runs = layout.get(subject='001', task='condassoc', extension='nii.gz')
pprint(sub_runs)

# Notice that these files are stored as BIDSImageFiles, a custom class created for pybids.
# We generally just want one thing directly from these objects- the file path.
path = sub_runs[0].path
print()
print('The file path for the first file:')
print(path)

In [ ]:
# Let's get the metadata for one of the functional scans
func_file = sub_runs[0].path
pprint(layout.get_metadata(func_file))

### Exercise 1
Load the dataset located in `/scratch/cis-training/ds001491` and print the name of the file with the **extension** `'nii.gz'` for **echo** `1` of the `'images'` **task** of **subject** `'01'`.

### Exercise 2
Use `BIDSLayout.get_metadata` to determine the EchoTime for that file.

## NiBabel
[NiBabel](https://nipy.org/nibabel/) allows you to **read and write neuroimaging files**.

In [ ]:
import nibabel as nib

In [ ]:
# Grab a random nifti file from the dataset
f = layout.get(extension='nii.gz', task='condassoc')[0].path
print(f)

In [ ]:
# Load the file as an image object
img = nib.load(f)
print(type(img))

In [ ]:
# Each image has an affine matrix that contains information
# about the orientation and dimensionality of the data
print(img.affine)

In [ ]:
# The image header contains a lot of its own information
hed = img.header
print(type(hed))

# get_zooms() prints the voxel sizes and
# repetition time (fourth dimension) for the image
hed.get_zooms()

In [ ]:
# The data are a 3D-to-4D array
data = img.get_data()
print(type(data))
print(data.shape)

In [ ]:
# You can also create images from an affine matrix and a numpy array
random_data = np.random.random((96, 96, 42))
random_img = nib.Nifti1Image(random_data, img.affine)
print(type(random_img))

In [ ]:
# You can also save image objects
random_img.to_filename('example_random_image.nii.gz')

### Exercise 3
Compute the mean of the data from the loaded functional scan and generate a new 3D image with the mean data.

## Nilearn
[Nilearn](http://nilearn.github.io) is primarily for doing machine learning on fMRI data, but it also provides a wide range of functionality, including:
- Applying masks to data
- Visualizing fMRI data
- Simple denoising and analysis
- Performing simple math on images

In [ ]:
import nilearn as nl
from nilearn import plotting, image, masking

first_volume = image.index_img(f, 0)

#### nilearn.masking
This submodule can be used to create masks, apply masks to data, and unmask masked data.

In [ ]:
from nilearn import masking

In [ ]:
# Grab a random functional file from the dataset
f = layout.get(suffix='bold', extension='nii.gz')[0].path
print(f)
print('The shape of the data in the file is {}'.format(nib.load(f).shape))

In [ ]:
# Generate a general brain mask from the functional data
mask = masking.compute_epi_mask(f)

# Let's plot it against the first volume in the functional scan
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_roi(mask, bg_img=first_volume, axes=ax)
fig.show()

In [ ]:
# Apply the mask to the file
data = masking.apply_mask(f, mask)

print('Shape of the functional data: {}'.format(nib.load(f).shape))
print('Shape of the mask: {}'.format(mask.shape))
print('Number of voxels within mask: {}'.format(np.sum(mask.get_data())))
print('Shape of the masked data: {}'.format(data.shape))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(16, 5))
ax.imshow(data.T, aspect='auto')
ax.set_xlabel('Time')
ax.set_ylabel('Voxel')
fig.show()

In [ ]:
# Let's plot one voxel's time series
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(data[:, 0])
ax.set_xlabel('Time')
ax.set_ylabel('BOLD signal (arbitrary unit)')
ax.set_xlim(0, data.shape[0])
fig.show()

### nilearn.image
This submodule can perform operations (e.g., math, thresholding, resampling) on image files.

In [ ]:
from nilearn import image

In [ ]:
# We can use index_img to grab a specific
# volume within a 4D dataset
first_volume = image.index_img(f, 0)
print(first_volume.shape)

# Plot the first volume of the functional data
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_epi(first_volume, axes=ax, cut_coords=[0, 0, 20])
fig.show()

In [ ]:
# We can also compute the mean image
mean_img = image.mean_img(f)
print(mean_img.shape)

# Plot the mean image
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_epi(mean_img, axes=ax, cut_coords=[0, 0, 20])
fig.show()

In [ ]:
# Or apply a threshold
mean_img_thresh = image.threshold_img(mean_img, np.mean(masking.apply_mask(mean_img, mask)))

# Plot the first volume of the functional data
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_epi(mean_img_thresh, axes=ax, cut_coords=[0, 0, 20])
fig.show()

In [ ]:
# Or smooth an image
smooth_mean_img_thresh = image.smooth_img(mean_img_thresh, 8)

# Plot the first volume of the functional data
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_epi(smooth_mean_img_thresh, axes=ax, cut_coords=[0, 0, 20])
fig.show()

### Exercise 4
Use [nilearn.image.math_img](http://nilearn.github.io/modules/generated/nilearn.image.math_img.html#nilearn.image.math_img) to square the last volume in the functional dataset, then plot it.

### nilearn.datasets
Nilearn has the ability to download a range of datasets and useful images from the internet.

In [ ]:
from nilearn import datasets

In [ ]:
# Let's show what functions there are in this module
from inspect import getmembers, isfunction

functions_list = [o[0] for o in getmembers(datasets) if isfunction(o[1])]
print('\n'.join(functions_list))

In [ ]:
# We can fetch atlases
aal = datasets.fetch_atlas_aal()

# Let's plot this atlas
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_roi(aal['maps'], axes=ax, cut_coords=[0, 0, 20])
fig.show()

In [ ]:
# We can also fetch and plot *probabilistic* atlases, like MSDL
atlas = datasets.fetch_atlas_msdl()

# Let's plot this atlas
fig, ax = plt.subplots(figsize=(16, 6))
plotting.plot_prob_atlas(atlas['maps'], axes=ax, cut_coords=[0, 0, 20])
fig.show()

### Exercise 5
Fetch and plot the `'sub-maxprob-thr25-2mm'` version of the Harvard-Oxford atlas. Determine if it's a probabilistic atlas (atlas is a 4D image) or a deterministic one (atlas is a 3D image).
Hint: Use the [documentation](http://nilearn.github.io/modules/generated/nilearn.datasets.fetch_atlas_harvard_oxford.html#nilearn.datasets.fetch_atlas_harvard_oxford) to figure out what arguments are required.

## Other packages
Some other neuroimaging-related Python packages to know include:
- nistats: Modeling and GLMs for fMRI
- Nipype: Python wrappers for non-Python neuroimaging tools
- heudiconv: Automated conversion of scanner data to BIDS format
- DiPy: Diffusion-weighted imaging processing and analysis in Python